In [1]:
import torch
import cv2
import glob

In [2]:
cfg_model_path='best (2).pt'

In [3]:
model=torch.hub.load('ultralytics/yolov5', 'custom', path=cfg_model_path, force_reload=True)
model.to('cuda')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Admin/.cache\torch\hub\master.zip
YOLOv5  2023-12-16 Python-3.8.18 torch-2.1.0+cu118 CUDA:0 (Quadro P1000, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [6]:
#reading video file and resizing into 380x640 size and passing it to model
board_counter=0
cy1=320
offset=4.5
frame_counter=0
cap = cv2.VideoCapture('./vid/web - Copy.mp4')
while True:
    ret, frame = cap.read()
    frame_counter+=1
    if ret == False:
        break
    frame = cv2.resize(frame, (640, 380))
    frame_save=frame.copy()
    cv2.line(frame,(0,cy1),(640,cy1),(0,0,255),1)
    results = model(frame)
    a=results.pandas().xyxy[0]
    for index, row in a.iterrows():
        if row['class'] == 0:
            x1=int(row['xmin'])
            y1=int(row['ymin'])
            x2=int(row['xmax'])
            y2=int(row['ymax']) 

            rectx1,recty1=((x1+x2)/2),((y1+y2)/2)
            rectcenter=int(rectx1),int(recty1)
            cx=rectcenter[0]
            cy=rectcenter[1]
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.circle(frame,(cx,cy), 5, (0, 0, 255), -1)
            if cy>cy1-offset and cy<cy1+offset:
                board_counter+=1 
                #creating csv file to save frame number and board count
                with open('board_count.csv', 'a') as f:
                    f.write(str(board_counter)+','+str(frame_counter)+'\n')
                #creating a folder to save the frame with board count
                cv2.imwrite('./board_occ/'+str(board_counter)+'.jpg',frame_save)
                


    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()